In [ ]:
# Import lib
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
#from google.colab import files
import time
from glob import glob

In [ ]:
DATABASE = "/Users/Ombeline/Documents/IMT/M2/4 - UV Projet/nimages_test"

Fonction pour load les images dans un numpy à partir d'un subfolder

In [ ]:
def load_depth_images_from_folder(subfolder):
  depth_images = []
  for filename in sorted(os.listdir(subfolder)):
    if filename.endswith("_D.png"):
      img = cv2.imread(os.path.join(subfolder, filename), cv2.IMREAD_UNCHANGED)
      depth_images.append(img)
  return np.array(depth_images)

# 1) Trouver la taille de la séquence la plus petite

In [ ]:
def find_prefix_vid(global_fold):
  """
    This function searches for unique prefixes of folders in the global_fold directory.
  """
  prefixes_to_process = set()

  for fold in os.listdir(global_fold):
    dossier_path = os.path.join(global_fold, fold)
    if os.path.isdir(dossier_path):
        # Extract folder prefix
        prefix = fold.split("_", 1)[0] + "_" + fold.split("_", 1)[1].split("_")[0]
        prefixes_to_process.add(prefix)

  return prefixes_to_process

In [ ]:
def find_min_length_seq(video_folds_paths):
  """
  This function return the length of the smallest sequence in a video (depth)
  """
  min_seq = 10000
  for fold in video_folds_paths:
    if min_seq > len(os.listdir(fold)):
      min_seq = len(os.listdir(fold))

  return int(min_seq / 2)

# 2) Trouver le nbr d’images à supprimer

In [ ]:
def find_nb_to_delete(subfold_path, min_length):
  number_to_delete = (len(os.listdir(subfold_path)) / 2) - min_length
  return int(number_to_delete)

# 3) Trouver la frame centrale

In [ ]:
def select_frame(subfold_path):
  """
  Select the best frame from alls, based on the maximum height
  The number returned is the position of the frame in the list (starting by 1)
  """
  depth_images = load_depth_images_from_folder(subfold_path)

  # Initialiser les variables pour la profondeur maximale et la position correspondante
  profondeur_max = 10000
  image_position_max = 10000
  # Parcourir toutes les images dans le dossier
  for i, depth_image in enumerate(depth_images, start=1):
    depth_non_zero = depth_image[depth_image != 0]
    # Vérifier si l'image est valide (non nulle)
    if depth_non_zero is not None:
      # Trouver la profondeur maximale dans l'image
      profondeur_max_image = depth_non_zero.min()

      # Mettre à jour si la profondeur maximale est plus grande que celle précédemment trouvée
      if profondeur_max_image < profondeur_max:
        profondeur_max = profondeur_max_image
        image_position_max = i

  return image_position_max

# 4) Compter la taille de la 1ère et 2eme portion de séquence

In [ ]:
def count_before_after(subfold_path, frame_center_idx):
  subfold = os.listdir(subfold_path)
  length = len(subfold) / 2
  nb_before, nb_after = frame_center_idx - 1, length - frame_center_idx
  return int(nb_before), int(nb_after)

# 5) Compter le nombre d'images à supprimer dans 1ère et 2ème portion

In [ ]:
# Fonction pour calculer portion avant et après centre
def count_delete_before_after(nb_to_delete, nb_before, nb_after, subfold_path):
  length_subfold = len(os.listdir(subfold_path))
  length_depth_subfold = length_subfold/2
  nb_to_delete_before = round((nb_before / (length_depth_subfold - 1)) * nb_to_delete)
  nb_to_delete_after = round((nb_after / (length_depth_subfold - 1)) * nb_to_delete)
  return nb_to_delete_before, nb_to_delete_after

# 6) Calculer la fréquence de suppression des images dans la 1ère et la 2ème portion

In [ ]:
# Calculer la fréquence d’images à supprimer
def freq_to_delete(nb_before, nb_after, nb_to_delete_before, nb_to_delete_after):

  if nb_to_delete_before != 0 and nb_to_delete_after != 0:
    freq_before = nb_before / nb_to_delete_before
    freq_after = nb_after / nb_to_delete_after

  elif nb_to_delete_before == 0 and nb_to_delete_after == 0:
      freq_before = 0
      freq_after = 0

  elif nb_to_delete_before == 0:
    freq_before = 0
    freq_after = nb_after / nb_to_delete_after

  else:
     freq_before = nb_before / nb_to_delete_before
     freq_after = 0

  return freq_before, freq_after

# 7) Déterminer la liste des frame à supprimer

In [ ]:
def find_images_to_delete(subfold_path, freq_before, freq_after, frame_center_idx):
  """
  This function is returning a list of the position of the images to delete in a subfolder (Depth + RGB)
  """
  depth_images = load_depth_images_from_folder(subfold_path)
  size_before = len(depth_images[:frame_center_idx-1])
  size_after = len(depth_images[frame_center_idx:])

  list_position_depth, list_position_image = [], []
  sum_before, sum_after = 0, len(depth_images) + 1

  if freq_before != 0:
    for i in range(int(size_before/freq_before)):
      sum_before += freq_before
      list_position_depth.append(round(sum_before))

  if freq_after != 0:
    for i in range(int(size_after/freq_after)):
      sum_after -= freq_after
      list_position_depth.append(round(sum_after))

  for i in list_position_depth:
    list_position_image.append(i*2 - 1)
    list_position_image.append(i*2)

  return sorted(list_position_image)

In [ ]:
path_global_images = DATABASE
list_prefix_vid = find_prefix_vid(path_global_images)

# Iterating on each video
for prefix in list_prefix_vid:

  list_folds_prefix = os.path.join(path_global_images, f"{prefix}*")    # list_folds_prefix is list of prefixes ('VID_001') the discrimine each video in the global folder
  video_folds_path = sorted(glob(list_folds_prefix))    # 'video_folds' represents every folder of 1 video

  # 1) Find the length of the smallest subfolder sequence in a video
  min_length = find_min_length_seq(video_folds_path)

  for subfold_path in video_folds_path:
    # 2) Find the number of depth images that has to be deleted in the subfolder
    nb_to_delete = find_nb_to_delete(subfold_path, min_length)
    # 3) Find the perfect frame where the subject is right down the camera
    frame_center_position = select_frame(subfold_path)
    # 4) Find the number of depth images that are before and after the center frame
    nb_before, nb_after = count_before_after(subfold_path, frame_center_position)
    # 5) Compute the number of depth images to delete before and after the center frame
    nb_to_delete_before, nb_to_delete_after = count_delete_before_after(nb_to_delete, nb_before, nb_after, subfold_path)
    # 6) Compute the deletion frequency of depth images before and after the center frame
    freq_before, freq_after = freq_to_delete(nb_before, nb_after, nb_to_delete_before, nb_to_delete_after)
    # 7) Create the list with the positions of depth and RGB images to delete in a subfolder
    list_images_to_delete = find_images_to_delete(subfold_path, freq_before, freq_after, frame_center_position)

    # 8) Delete the images in the subfolder corresponding to the past list
    for i, file in enumerate(sorted(os.listdir(subfold_path)), start=1):
      file_path = os.path.join(subfold_path, file)
      if i in list_images_to_delete:
        os.remove(file_path)

In [ ]:
list_prefix_vid = find_prefix_vid(path_global_images)

# Iterating on each video
for prefix in list_prefix_vid:
    list_folds_prefix = os.path.join(path_global_images, f"{prefix}*")    # list_folds_prefix is list of prefixes ('VID_001') the discrimine each video in the global folder
    video_folds_path = sorted(glob(list_folds_prefix))    # 'video_folds' represents every folder of 1 video

    min_length = find_min_length_seq(video_folds_path)

    for subfold_path in video_folds_path:
        if (len(os.listdir(subfold_path))/2 > min_length):
            print("Attention, le dossier suivant contient plus d'images que la longueur minimale : ", subfold_path)